# Discussion Week 2 - Introduction to Pandas and Scikit-Learn

[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tools4ds/fa2024/blob/main/static_files/discussion_slides/student_performance.ipynb)

- This Jupyter Notebook will brief you with an introduction to Pandas and Scikit-Learn, as we get our hands dirty on a dataset.
- We will first take a look at how to view and understand the dataset, followed by some preprocessing, exploratory data analysis and visualizations of various trends in the dataset.
- We then will dive into an example of building a Machine Learning model that explores the relationship between various factors specified in the dataset and predict the 'student exam scores'. 
- The aim is to understand these relationships and predict exam scores using a ML model (scikit-learn)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

### Let's begin by first reading the CSV file that contains the dataset.

In [ ]:
df = pd.read_csv('StudentPerformanceFactors.csv')

#### We can then take a look at the shape of the dataset.

In [ ]:
df.shape

In [ ]:
df.describe()

#### In order to view the first 5 records of the dataset, we need to use the .head() function, and .tail() to view the last 5 records

In [ ]:
df.head()

#### To drop null values present in the dataset, we can use the .dropna() function.

In [ ]:
df.info()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

#### Let's take a look at the data types present in the dataset. 

In [ ]:
df.dtypes

In [ ]:
df.columns

#### Pandas allows you to reference a column similar to a python dictionary key, using column names in square brackets.
#### This returns a Series object, the other fundamental data structure in Pandas.

In [ ]:
type(df['Hours_Studied'])  

#### To just view the top 5 records of a particular column we can do as following

In [ ]:
df['Hours_Studied'].head()

In [ ]:
df[['Attendance', 'Parental_Involvement']].head()

#### To view the unique values along with their counts in a particular column, we can do as follows:

In [ ]:
df['Access_to_Resources'].value_counts()

#### Retrieving a row of the DataFrame using integer-based indexing

In [ ]:
df.iloc[0]

#### Converting attendance to a fraction of 100

In [ ]:
df['Attendance'] = df['Attendance']/100

#### Adding a new column is as simple as shown below

In [ ]:
df['study_efficiency'] = df['Exam_Score'] / df['Hours_Studied']

In [ ]:
df.head()

#### Deleting a row/column are shown in the following cells

In [ ]:
#dropping a column
df.drop('Tutoring_Sessions', axis=1, inplace=True)

In [ ]:
#dropping a row (given an index)
df.drop(0)

#### Filtering a dataframe based on a condition

In [ ]:
low_parental_involvement = df[df['Parental_Involvement'] == 'Low']
high_scorers = df[df['Exam_Score'] > 75]

In [ ]:
low_parental_involvement.head()

In [ ]:
high_scorers.head()

#### If we want to sort the dataframe in a particular order (ascending/descending) based on a particular column, we can do as follows

In [ ]:
sorted_exam_score = df.sort_values(by='Exam_Score', ascending=False)

In [ ]:
sorted_exam_score.head()

#### Using groupby, is a very powerful way of grouping 2 or more columns together and applying aggreagte functions to each group. It aids in the EDA process uncovering some really important insights pertaining to the dataset.
#### It is widely used in data analysis to summarize, filter, and transform data by grouping similar values.

In [ ]:
#average exam score by gender
df.groupby('Gender')['Exam_Score'].mean()

In [ ]:
#median average score by teacher quality
df.groupby('Teacher_Quality')['Exam_Score'].median()

#### We can apply a transformation after grouping few columns and use the lambda function to perform some computation, in this case we standardize the values of Exam Scores.

In [ ]:
df.groupby('Family_Income')['Exam_Score'].transform(lambda x: (x-x.mean())/x.std())

#### Let's plot some results!!
#### We can use a combination of matplotlib and seaborn to plot some nice visualization of different statistical findings.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

- #### Matplotlib is a versatile and powerful Python library used for creating static, interactive, and animated visualizations. (line plots, histograms, scatter plots)
- #### Seaborn is like Matplotlib but with an extra flair—pre-styled, easy-to-use visualizations like heatmaps, violin plots, and pair plots are its specialty. Seaborn takes care of aesthetics so you can focus on your data.

In [ ]:
df['Family_Income'].value_counts()

In [ ]:
df['Standardized_Scores'] = df.groupby('Family_Income')['Exam_Score'].transform(lambda x: (x-x.mean())/x.std())

# Plot the standardized scores by Family Income
plt.figure(figsize=(10, 6))
sns.boxplot(x='Family_Income', y='Standardized_Scores', data=df)
plt.title('Distribution of Standardized Exam Scores by Family Income')
plt.xlabel('Family Income')
plt.ylabel('Standardized Exam Scores')
plt.show()

- #### The **IQR (Interquartile Range)** is the difference between the third quartile (Q3, the 75th percentile) and the first quartile (Q1, the 25th percentile). So, `IQR = Q3 - Q1`.
- #### The whiskers represent the range of data points that are not considered outliers
- #### The lower whisker extends to the smallest data point that is **at least** `Q1 - 1.5 * IQR`.
- #### The upper whisker extends to the largest data point that is **at most** `Q3 + 1.5 * IQR`.

In [ ]:
average_scores_gender = df.groupby('Gender')['Exam_Score'].mean().reset_index()

# Creating a bar plot
plt.figure(figsize=(8, 5))
sns.barplot(x='Gender', y='Exam_Score', data=average_scores_gender)
plt.title('Average Exam Scores by Gender')
plt.xlabel('Gender')
plt.ylabel('Average Exam Score')
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.scatterplot(x='Hours_Studied', y='Exam_Score', data=df, hue='Gender', style='Gender', s=50)
plt.title('Relationship Between Hours Studied and Exam Score')
plt.xlabel('Hours Studied')
plt.ylabel('Exam Score')
plt.legend(title='Gender')
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(x='Parental_Involvement', data=df, palette='autumn')
plt.title('Count of Parental Involvement Levels')
plt.xlabel('Parental Involvement')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()

## Onto some Machine Learning!

- #### Let's build a linear regression using sklearn

- #### First we need to handle the categorical variables using One-Hot Encoding

- #####  One-hot encoding is a technique used to convert categorical data into a numerical format by representing each category as a binary vector. 
- ##### Each unique category is transformed into a new column, and for a given row, only the column corresponding to the category is marked as 1, while the rest are set to 0.

In [ ]:
df.drop('study_efficiency', axis=True, inplace=True)

In [ ]:
cat_variables = df.select_dtypes(include=['object']).columns.tolist()
df_processed = pd.get_dummies(df, columns=cat_variables)

In [ ]:
df_processed.head()

####  We can do the same as above using OneHotEncoder of the sklearn library

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False)  
encoded_data = ohe.fit_transform(df[cat_variables])

In [ ]:
encoded_df = pd.DataFrame(encoded_data, columns=ohe.get_feature_names_out(cat_variables))

#### Joining the non categorical variable from the original to the encoded df

In [ ]:
complete_encoded_df = pd.concat([df.drop(cat_variables, axis=1), encoded_df], axis=1)   

In [ ]:
encoded_df.head()

In [ ]:
complete_encoded_df.head()

In [ ]:
complete_encoded_df.dropna(inplace=True)

#### In this example, we're going to try and build a Exam Score predictor based on various features.
- ##### Use all the columns except the Exam Score column as predictors (features) for the linear regression model.
- ##### The Exam Score is the target variable, which we want to predict.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

X = complete_encoded_df.drop('Exam_Score', axis=1)
y = complete_encoded_df['Exam_Score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(y_test.shape)
print(y_train.shape)


# Linear Regression

- Linear regression is a statistical method used to model the relationship between a dependent variable and one or more independent variables. 
- The goal is to find the line (or hyperplane in higher dimensions) that best fits the data points by minimizing the differences between predicted and actual values.

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
y_preds = model.predict(X_test)

- **Mean Squared Error (MSE)** is the average of the squared differences between the predicted values and the actual values. It is calculated using the formula:

  $$\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$$

  where:
  - `y_i`: actual value
  - `ŷ_i`: predicted value
  - `n`: number of data points

- **Root Mean Squared Error (RMSE)** is the square root of the MSE. It provides the error in the same units as the original data and is calculated using the formula:

  $$\text{RMSE} = \sqrt{\text{MSE}} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}$$

- **Mean Absolute Error (MAE)** is the average of the absolute differences between the predicted and actual values. It is calculated using the formula:

  $$\text{MAE} = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|$$


In [ ]:
mse = mean_squared_error(y_test, y_preds)
print(mse)

In [ ]:
rmse = mean_squared_error(y_test, y_preds, squared=False)
print(rmse)

In [ ]:
mae = mean_absolute_error(y_test, y_preds)
print(mae)

### Challenge for you:
#### The Mean Absolute Error is pretty good so far but, do you think we can improve it any further using any statistical techniques?

In [ ]:
from sklearn.preprocessing import StandardScaler
X = df.drop(columns='Exam_Score')
y = df['Exam_Score']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

categorical_columns = X_train.select_dtypes(include=['object']).columns

# Scale numerical columns
scaler = StandardScaler()
numerical_columns = X_train.select_dtypes(
    include=['int64', 'float64']).columns

X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[numerical_columns] = scaler.fit_transform(
    X_train[numerical_columns])
X_test_scaled[numerical_columns] = scaler.transform(
    X_test[numerical_columns])

In [ ]:
X_train_correct = pd.get_dummies(
X_train_scaled, columns=categorical_columns)
X_test_correct = pd.get_dummies(X_test_scaled, columns=categorical_columns)

In [ ]:
model_new = LinearRegression()
model_new.fit(X_train_correct, y_train)

In [ ]:
y_preds = model_new.predict(X_test_correct)

In [ ]:
mae = mean_absolute_error(y_test, y_preds)
print(mae)